In [5]:
class A:
    def __init__(self, a):
        self.a = a 
    
    def __getattr__(self, attr):
        if attr in self.__dict__(attr):
            return self.__dict__[attr]
        raise AttributeError
    
    def __setattr__(self, attr, value):
        print('Setting attr', attr)
        if attr == 'a':
            self.__dict__[attr] = value
        else:
            raise AttributeError

In [6]:
a = A(1)

Setting attr a


In [7]:
a.a = 1 # по идее этот метод не должен запускаться

Setting attr a


In [8]:
a.b = 3

AttributeError: 

In [9]:
A.__dict__ # содержит все свои атрибуты

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.A.__init__(self, a)>,
              '__getattr__': <function __main__.A.__getattr__(self, attr)>,
              '__setattr__': <function __main__.A.__setattr__(self, attr, value)>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [10]:
a.__dict__

{'a': 1}

In [15]:
b = A(2) # wtf

Setting attr a


вместо словаря можно использовать слоты:

In [36]:
class Slots:
    __slots__ = ('a',) # ограничивает возможность заводить атрибуты
    def __init__(self, a):
        self.a = a

In [37]:
s = Slots(1)

In [18]:
s.a

1

In [21]:
s.__dict__ # slots перезаписывает словарь

AttributeError: 'Slots' object has no attribute '__dict__'

In [24]:
class Child(Slots):
    pass

In [25]:
c = Child(1)

In [26]:
c.s = 1

In [27]:
c.__dict__ # а тут словарь появился

{'s': 1}

In [29]:
class Child(Slots):
    __slots__ = ('b',)
    def __init__(self, a, b):
        self.a = a
        self.b = b

In [30]:
c = Child(1, 2)

In [32]:
c.c = 3

AttributeError: 'Child' object has no attribute 'c'

In [52]:
def get_set_delete(obj):
    obj.a
    obj.a = 1
    del obj.a

In [34]:
get_set_delete(c)

In [35]:
c.a

AttributeError: 'Child' object has no attribute 'a'

----------------------------------------

In [56]:
class Slots:
    __slots__ = ('a',) # ограничивает возможность заводить атрибуты
    def __init__(self, a):
        self.a = a

In [57]:
s = Slots(1)

In [58]:
s.a

1

In [60]:
class NoSlots:
    def __init__(self, a):
        self.a = a

In [61]:
n = NoSlots(1)

In [62]:
n.a

1

In [ ]:
%timeit get_set_delete(s)

In [ ]:
%timeit get_set_delete(n)

In [64]:
# __attr => ClassName__attr
# __getattr__, __setattr__
# __slots__
# __setatribute__, __getattribute__
# descriptor

In [ ]:
class A:
    def __init__(self, a):
        self.a = a 
    
    def __getattribute__(self, attr):
        print('Getting attr', attr)
        if attr in self.__dict__(attr):
            return self.__dict__[attr]
        raise AttributeError
    
    def __setattribute__(self, attr, value):
        print('Setting attr', attr)
        if attr == 'a':
            self.__dict__[attr] = value
        else:
            raise AttributeError

In [76]:
# propetry

class Person:
    def __init__(self):
        self.__name = '' # скрытый метод
    
    def getname(self):
        return self.__name
    
    def setname(self, value):
        if not value.isalpha():
            raise AttributeError
        self.__name = value
    
    def delname(self):
        del self.__name
    
    name = property(getname, setname, delname) # статический метод

In [77]:
p = Person()

In [78]:
p.name

''

In [79]:
p._Person__name

''

In [80]:
p.name = 'Vasya'

In [81]:
p.name # вызывается стат.метод name

'Vasya'

In [69]:
p.name = '1234'

AttributeError: 

In [8]:
class Name:
    def __set_name__(self, owner, name): # вызывается при создании атрибута класса, который является дескриптором
        print('Set_name')
        self.private = '__' + name
        self.public = name
    
    def __get__(self, instance, owner): # вызывается, когда происходит доступ к атрибуту класса
        print('Getname')
        return getattr(instance, self.private) # достает приватное значение атрибута
    
    def __set__(self, instance, value): # вызывается, когда мы пытаемся присвоить атрибуту какое-то значение
        print('Setnamevalue')
        if isinstance(value, str) and value.isalpha():
            setattr(instance, self.private, value) # если введено корректное значение, оно присваивается приватному атрибуту
        else:
            raise AttributeError


In [9]:
class Person:
    name = Name() # атрибут класса - дескриптор

    def __init__(self, name, surname, age):
        self.name = name # при присвоении этого атрибута вызывается дескриптор
        self.surname = surname
        self.age = age

Set_name


In [10]:
p = Person('Vasya', 'Pupkin', 10)

Setnamevalue


In [11]:
p.name

Getname


'Vasya'

: 

In [96]:
vars(p)

{'__name': 'Vasya', 'surname': 'Pupkin', 'age': 10}

In [93]:
p.name = '123'

Setnamevalue


AttributeError: 

In [1]:
class Descriptor:
    def __get__(self, instance, owner):
        return 'значение'

class MyClass:
    attr = Descriptor()

In [2]:
my_object = MyClass()

In [3]:
my_object.attr  # выведет 'значение'

'значение'

In [4]:
MyClass.attr

'значение'

#### Методы класса и статические методы

In [104]:
from math import pi

class Pizza:
    def __init__(self, *ingrs):
        self.ingredients = list(ingrs)
    
    def price(self):
        return len(self.ingredients)
    
    def pizza_area(cls, d):
        return f"{cls.__name__}: {pi * (d / 2) ** 2}"
    
    # pizza_area = staticmethod(pizza_area)
    pizza_area = classmethod(pizza_area)

In [105]:
p = Pizza('pepperoni', 'cheese', 'mayonnaise')
p.pizza_area(30) # можем вызвать его, даже если экземпляра нет

'Pizza: 706.8583470577034'

In [ ]:
# dict.fromkeys() - тоже метод класса